<span>
<b>Python version:</b>  >=3.7<br/>
<b>Networkx version:</b>  >=2.3<br/>
<b>Last update:</b> 24/11/2021
</span>

<a id='top'></a>
# *Chapter 3: Ties Strength & Resilience*

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started. 

In [ ]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Reading Game of Thrones Season 6 edge data and defining the graph g (useful for further operations).

Download data from [here](https://drive.google.com/file/d/1NIS_LCye-SgwP6UM6JeNHST5PWR7ZwwR/view?usp=sharing).

`weight` is the number of interactions between the characters.



In [ ]:
def read_net_w(filename):
    g = nx.Graph()
    with open(filename) as f:
        f.readline()
        for l in f:
            l = l.split(",")
            g.add_edge(l[0], l[1], weight=int(l[2]))
    return g

# Game of Thrones data
season = 6
g = read_net_w(f'asioaf/got-s{season}-edges.csv')

In [ ]:
nx.spring_layout(g)
fig = plt.figure(1, figsize=(80, 40), dpi=60)
nx.draw_spring(g, with_labels = True, font_size=30, width=2, node_size=1000,node_color='#A0CBE2')

In [ ]:
for e in g.edges(data=True):
    print(e)
    break

## Bridges and Local Bridges

Checking if graph has bridges

In [ ]:
nx.has_bridges(g)

Generating all bridges in the graph

In [ ]:
list(nx.bridges(g))

Generating all local bridges in the graph and computing the span (i.e., the shortest path length between the endpoints if the local bridge is removed)

In [ ]:
list(nx.local_bridges(g, with_span=True ))

## Tie Strength

Measuring Tie Strength for each pair of nodes in the graph through Neighborhood Overlap

In [ ]:
def node_overlap(g):
    for u, v in g.edges():
        n_u = set(g.neighbors(u)) # set of u neighbors
        n_v = set(g.neighbors(v)) # set of v neighbors
        # & is intersection, | is union
        overlap = len(n_u & n_v) / len(n_u | n_v) # Neighborhood Overlap
        g[u][v]['overlap'] = overlap
    return g

In [ ]:
g = node_overlap(g)

In [ ]:
for e in g.edges(data=True):
    print(e)
    break

Plotting the KDE (Kernel Density Estimation) of Neighborhood Overlap

In [ ]:
weights = [e[-1]['overlap'] for e in g.edges(data=True)] 
pd.DataFrame(weights)[0].plot.kde()
plt.xlabel("Neighborhood Overlap")
plt.xlim(0,1)
plt.show()

Plotting the KDE (Kernel Density Estimation) of Interactions Weights

In [ ]:
weights_got = [d['weight'] for u,v,d in g.edges(data=True)]
pd.DataFrame(weights_got)[0].plot.kde()
plt.xlabel("Interaction Weights")
plt.xlim(0,max(weights_got))
plt.show()

### Exercises

- Load GOT season 1 and replicate the computations and plots above. What are the bridges and local bridges? 
- Compare GOT season 1 with GOT season 5

## Network Resilience
In the following we:
- generate Random and Scale-free Network
- compute Reslience for both Networks (through molloy_reed and breakdown_threshold)
- plot Game of Thrones graph Resilience under Random Failure and Targeted Attacks

Generating Networks

In [ ]:
# Random
er = nx.erdos_renyi_graph(1000, 0.001)
# Scale-free
ba = nx.barabasi_albert_graph(1000, 2)

Computing node degree for both ER and BA Networks

In [ ]:
er_degs = list(dict(er.degree()).values())
ba_degs = list(dict(ba.degree()).values())

### Defining Molloy-Reed threshold

The mathematical derivation for the threshold at which a complex network will lose its giant component is based on the Molloy–Reed criterion.

The Molloy–Reed criterion is derived from the basic principle that in order for a giant component to exist, on average each node in the network must have at least two links. This is analogous to each person holding two others' hands in order to form a chain.

Using this criterion and an involved mathematical proof, one can derive a critical threshold for the fraction of nodes needed to be removed for the breakdown of the giant component of a complex network.

In [ ]:
def molloy_reed(degrees):
    return (np.mean(degrees)*(np.mean(degrees)+1))/np.mean(degrees)

def breakdown_threshold(degrees):
    K = molloy_reed(degrees)
    return 1 - (1/(K-1))

Random network

In [ ]:
molloy_reed(er_degs)

In [ ]:
breakdown_threshold(er_degs) * 100

Scale-free network

In [ ]:
molloy_reed(ba_degs)

In [ ]:
breakdown_threshold(ba_degs) * 100

### Random Failures

In [ ]:
def random_node(g): # select a random node from graph
    return [np.random.choice(g.nodes())]

def dismantle(g, function, **args): # incrementally removes node from a graph and computes size of connected components
    total_nodes = g.number_of_nodes()
    removed_nodes = []
    components = []
    while len(g.nodes()) > 1:
        n = function(g, **args)[0]
        g.remove_node(n)
        removed_nodes.append((len(removed_nodes)+1)/total_nodes)
        comps = list(nx.connected_components(g))
        g_size = 0
        if len(comps)>0:
            g_size  = max([len(c)for c in comps])/total_nodes
        components.append(g_size)
    return removed_nodes, components

def get_sorted_nodes(g, score, reverse=True): # sort nodes
    nodes = score(g)
    if isinstance(nodes, dict):
        nodes = [(k, v) for k, v in nodes.items()]
    srt = sorted(nodes, key = lambda k: k[1], reverse = reverse)
    return [x[0] for x in srt]

def plot_dismantle(x, y):
    plt.plot(x, y)
    plt.xlabel("Removed Nodes")
    plt.ylabel("Giant Component size")
    plt.show()

Random Failure:
- Giant component size has a consistent decrease when a big fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, random_node)
plot_dismantle(rn, comps)

### Targeted Attacks

Targeted Node attack (hubs are removed first)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.degree)
plot_dismantle(rn, comps)

Targeted Edge Attack (based on Edge Betweenness Centrality)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.betweenness_centrality)
plot_dismantle(rn, comps)

Targeted Edge Attack (based on Edge Harmonic Centrality)
- Giant component size has a consistent decrease when a small fraction of nodes is removed

In [ ]:
h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.harmonic_centrality)
plot_dismantle(rn, comps)

### Putting all together

In [ ]:
h = g.copy()
rn, comps = dismantle(h, random_node)
plt.plot(rn, comps, label='random')

h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.degree)
plt.plot(rn, comps, label='by degree')

h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.betweenness_centrality)
plt.plot(rn, comps, label='by betweenness')

h = g.copy()
rn, comps = dismantle(h, get_sorted_nodes, score=nx.harmonic_centrality)
plt.plot(rn, comps, label='by harm centrality')

plt.xlabel("Removed Nodes")
plt.ylabel("Giant Component size")
plt.legend()
plt.show()